In [ ]:
import math
from scipy.stats import norm

print("Welcome to the European option pricer. Input 'call' or 'put' and relevant details as you follow.")

def BSM(K, S, T_days, r, IV, option_type):
    T = T_days / 365  # convert time to years

    d1 = (math.log(S / K) + (r + 0.5 * IV**2) * T) / (IV * math.sqrt(T))
    d2 = d1 - IV * math.sqrt(T)

    pdf_d1 = norm.pdf(d1)

    if option_type.lower() == "call":
        price = S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
        delta = norm.cdf(d1)
        theta = (-S * pdf_d1 * IV / (2 * math.sqrt(T))
                 - r * K * math.exp(-r * T) * norm.cdf(d2))
        rho = K * T * math.exp(-r * T) * norm.cdf(d2)

    elif option_type.lower() == "put":
        price = K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        delta = norm.cdf(d1) - 1
        theta = (-S * pdf_d1 * IV / (2 * math.sqrt(T))
                 + r * K * math.exp(-r * T) * norm.cdf(-d2))
        rho = -K * T * math.exp(-r * T) * norm.cdf(-d2)

    else:
        raise ValueError("Option type must be 'call' or 'put'")

    gamma = pdf_d1 / (S * IV * math.sqrt(T))
    vega = S * pdf_d1 * math.sqrt(T)

    # Adjust units:
    theta_daily = theta / 365          # Theta per day
    vega_per_1pct = vega / 100         # Vega per 1% volatility change
    rho_adj = rho / 100                # Rho adjusted to fit convention


    return price, {
        "Delta": delta,
        "Gamma": gamma,
        "Vega": vega_per_1pct,
        "Theta": theta_daily,
        "Rho": rho_adj
    }


# ---- User input ----
option_type = input("Enter option type ('call' or 'put'): ")

K = float(input("Enter Strike/Exercise Price: "))
S = float(input("Enter the underlying asset value/stock price: "))
T_days = float(input("Enter time till expiration in days: "))
r = float(input("Enter the risk free return (e.g. 5% = 0.05): "))
IV = float(input("Enter Implied Volatility (e.g. 20% = 0.2): "))

# ---- Calculate ----
try:
    price, greeks = BSM(K, S, T_days, r, IV, option_type)
    print(f"\nThe {option_type} option price is: {price:.2f}")
    print("Greeks:")
    for g, v in greeks.items():
        print(f"{g}: {v:.6f}")

except Exception as e:
    print(f"An error occurred: {e}")
